# 청와대 청원 데이터 분석하기 

## 웹크롤링

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
titles = []
start_date = []
end_date = []
num_consent = []
answer_date = []
address = []

def get_info(url): 
    res = urlopen(url)
    soup = BeautifulSoup(res, "html.parser")
    
    # get titles, address(url)
    contents = soup.find_all('div', class_='ans_name_title')
    for n in range(0,len(contents)):
        content = contents[n]
        titles.append(content.a.get_text())
        add_url = str(content.find('a')['href'])
        address.append('https://www1.president.go.kr'+add_url)
    
    # 청원시작과 끝 날짜, 참여인원, 답변일 정보 얻기 
    infos = soup.find_all('ol', class_='ans_name_info')
    for info in infos: 
        start_date.append(info.find('p', class_='ans_list1_2').get_text()[2:-2])
        end_date.append(info.find('p', class_='ans_list2_2').get_text()[2:-2])
        num_consent.append(info.find('p', class_='ans_list3_2').get_text()[2:-2])
        answer_date.append(info.find('p', class_='ans_list4_2').get_text()[2:-2])          

In [3]:
for n in range(1,13): 
    url = 'https://www1.president.go.kr/petitions/answer?page=%d' % n
    get_info(url)

In [4]:
data = {'제목':titles, '청원시작일':start_date, '청원종료일': end_date,
       '참여인원': num_consent, 'URL':address}
df = pd.DataFrame(data)
df

,제목,청원시작일,청원종료일,참여인원,URL
0,방송통신위원회는 방송의 공적 책임 방기하고 언론이기를 포기한 채널A와 TV조선의 ...,2020. 04. 07,2020. 05. 07,"273,513",https://www1.president.go.kr/petitions/587827
1,렌트카 훔쳐 사망사고를 낸 10대 엄중 처벌해주세요,2020. 04. 02,2020. 05. 02,"1,007,040",https://www1.president.go.kr/petitions/587624
2,세월호 전면재수사,2020. 04. 01,2020. 05. 01,"216,118",https://www1.president.go.kr/petitions/587604
3,자가격리를 어기고 제주도 4박5일 여행.미국유학생 강남구 **번 확진자 처벌해주세요,2020. 03. 27,2020. 04. 26,"207,563",https://www1.president.go.kr/petitions/587334
4,수출용 코로나19 진단키트 이름을 독도로 해주세요,2020. 03. 25,2020. 04. 24,"385,617",https://www1.president.go.kr/petitions/587169
...,...,...,...,...,...
169,"권역외상센터 (이국종 교수님) 추가적, 제도적, 환경적, 인력 지원",2017. 11. 17,2017. 12. 17,"281,985",https://www1.president.go.kr/petitions/45581
170,조두순 출소반대,2017. 09. 06,2017. 12. 05,"615,354",https://www1.president.go.kr/petitions/10949
171,'주취감형(술을 먹으면 형벌 감형)' 폐지를 건의(청원)합니다.,2017. 11. 04,2017. 12. 04,"216,774",https://www1.president.go.kr/petitions/26024
172,낙태죄 폐지와 자연유산 유도약(미프진) 합법화 및 도입을 부탁드립니다.,2017. 09. 30,2017. 10. 30,"235,372",https://www1.president.go.kr/petitions/18278


In [5]:
df.to_csv('BlueHouse Petition data.csv', sep=',', encoding='utf-8')

In [6]:
import pandas as pd

petition = pd.read_csv('BlueHouse Petition data.csv', thousands=',', encoding='utf-8', index_col=0)
petition.head()

,제목,청원시작일,청원종료일,참여인원,URL
0,방송통신위원회는 방송의 공적 책임 방기하고 언론이기를 포기한 채널A와 TV조선의 ...,2020. 04. 07,2020. 05. 07,273513,https://www1.president.go.kr/petitions/587827
1,렌트카 훔쳐 사망사고를 낸 10대 엄중 처벌해주세요,2020. 04. 02,2020. 05. 02,1007040,https://www1.president.go.kr/petitions/587624
2,세월호 전면재수사,2020. 04. 01,2020. 05. 01,216118,https://www1.president.go.kr/petitions/587604
3,자가격리를 어기고 제주도 4박5일 여행.미국유학생 강남구 **번 확진자 처벌해주세요,2020. 03. 27,2020. 04. 26,207563,https://www1.president.go.kr/petitions/587334
4,수출용 코로나19 진단키트 이름을 독도로 해주세요,2020. 03. 25,2020. 04. 24,385617,https://www1.president.go.kr/petitions/587169


In [7]:
len(petition)

174

In [8]:
import re 
import time
from tqdm import tqdm_notebook
from urllib.request import urlopen
from bs4 import BeautifulSoup

category =[]
content = []
answer = []

for n in tqdm_notebook(petition.index): 
    url = urlopen(petition['URL'][n]) 
    if n / 2 == 0:
        time.sleep(1)
    else: 
        time.sleep(2)
    soup = BeautifulSoup(url, "html.parser") 
    
    # get category 
    sector = soup.find('ul', class_='petitionsView_info_list')
    try: 
        category.append(sector.li.get_text()[4:])
    except AttributeError:
        print('AttributeError: index-', n, 'URL-', petition['URL'][n])
    
    # get petition content 
    content_raw = soup.find_all('div', class_='View_write')
    try: 
        if len(content_raw) > 1:
            content_raw_text = content_raw[1].get_text()
        else: 
            content_raw_text = content_raw[0].get_text()   
    except IndexError: 
        print('IndexError:', petition['URL'][n])
    
    content_split = re.split('\r|\t|\n|\xa0', content_raw_text)
    content.append(''.join(content_split))

    # get petition answer 
    answer_raw = soup.find_all('div', class_='pr_tk25')
    if len(answer_raw) > 1: 
        answer_raw_text = re.split("\r|\t|\n|\xa0", answer_raw[1].get_text())
    else: 
        answer_raw_text = re.split("\r|\t|\n|\xa0", answer_raw[0].get_text())
    answer_raw_text = ' '.join(answer_raw_text)
    answer.append(answer_raw_text[7:]) 

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [9]:
print(len(category), len(content), len(answer))

174 174 174


In [10]:
petition['카테고리'] = category
petition['청원내용'] = content
petition['청원답변'] = answer

In [11]:
petition[petition['청원내용'] == '']

,제목,청원시작일,청원종료일,참여인원,URL,카테고리,청원내용,청원답변
144,합정 **** 불법 누드촬영,2018. 05. 17,2018. 06. 16,201590,https://www1.president.go.kr/petitions/235102,인권/성평등,,
145,위장•몰래카메라 판매금지와 몰카범죄 처벌을 강화해주세요.,2018. 03. 23,2018. 04. 22,209494,https://www1.president.go.kr/petitions/173355,안전/환경,,
146,여성도 대한민국 국민입니다. 성별 관계없는 국가의 보호를 요청합니다.,2018. 05. 11,2018. 06. 10,419006,https://www1.president.go.kr/petitions/230552,인권/성평등,,
149,다산신도시 실버택배 비용은 입주민들의 관리비로 충당해야합니다.,2018. 04. 17,2018. 05. 17,302082,https://www1.president.go.kr/petitions/203181,행정,,


In [14]:
for n in [144, 145, 146, 149]: 
    url = urlopen(petition['URL'][n]) 
    soup = BeautifulSoup(url, "html.parser") 
    
    # get petition content 
    content_raw = soup.find_all('div', class_='View_write')
    content_raw_text = content_raw[2].get_text()   
    content_split = re.split('\r|\t|\n|\xa0', content_raw_text)
    petition['청원내용'][n] = ''.join(content_split)
    
    # get petition answer 
    answer_raw = soup.find_all('div', class_='pr_tk25')
    answer_raw_text = re.split("\r|\t|\n|\xa0", answer_raw[2].get_text())
    answer_raw_text = ' '.join(answer_raw_text)
    petition['청원답변'][n] = answer_raw_text[7:]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [17]:
petition[144:150]

,제목,청원시작일,청원종료일,참여인원,URL,카테고리,청원내용,청원답변
144,합정 **** 불법 누드촬영,2018. 05. 17,2018. 06. 16,201590,https://www1.president.go.kr/petitions/235102,인권/성평등,한 유명 유튜버의 사례저는 성범죄 피해자입니다. 꼭 한번만 제 이야기를 들어주세요....,<정혜승 뉴미디어 비서관> 안녕하십니까. 뉴미디어비서관 정혜승입니다. 오늘 11...
145,위장•몰래카메라 판매금지와 몰카범죄 처벌을 강화해주세요.,2018. 03. 23,2018. 04. 22,209494,https://www1.president.go.kr/petitions/173355,안전/환경,2012년 전국의 몰카범죄는 2400건 2015년에 들어 3배 이상 증가해 무려 7...,<정혜승 뉴미디어 비서관> 안녕하십니까. 뉴미디어비서관 정혜승입니다. 오늘 11...
146,여성도 대한민국 국민입니다. 성별 관계없는 국가의 보호를 요청합니다.,2018. 05. 11,2018. 06. 10,419006,https://www1.president.go.kr/petitions/230552,인권/성평등,최근 홍대 누드크로키 모델의 불법촬영 사건이 있었습니다.사건은 굉장히 빠르게 처리되...,<정혜승 뉴미디어 비서관> 안녕하십니까. 뉴미디어비서관 정혜승입니다. 오늘 11...
147,미세먼지의 위험 그리고 오염 및 중국에대한 항의,2018. 03. 24,2018. 04. 23,278128,https://www1.president.go.kr/petitions/174292,안전/환경,미세먼지가 10년전에 비해 상당히 자주 몰려오고있습니다.언론에서도 중국발이라고 얘기...,<정혜승 뉴미디어 비서관> 안녕하십니까. 뉴미디어비서관 정혜승입니다. 오늘 ...
148,GMO완전표시제 시행을 촉구합니다!,2018. 03. 12,2018. 04. 11,216886,https://www1.president.go.kr/petitions/164255,기타,우리나라는 안전성 논란이 계속되고 있는 식용 GMO를 연간 200만 톤 이상 수입합...,<정혜승 뉴미디어 비서관> - 안녕하세요. 뉴미디어비서관 정혜승입니다. 오늘 1...
149,다산신도시 실버택배 비용은 입주민들의 관리비로 충당해야합니다.,2018. 04. 17,2018. 05. 17,302082,https://www1.president.go.kr/petitions/203181,행정,다산신도시 입주민들이 택배원 대상으로 갑질을 저질러 사회적으로 큰 물의를 빚은 바 ...,<뉴미디어비서관> - 안녕하세요. 뉴미디어비서관 정혜승입니다. 오늘 11시 50분...


In [18]:
petition.to_csv('Final BlueHouse Petition data.csv', sep=',', encoding='utf-8')

### EDA

In [31]:
import pandas as pd
import platform
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

path = "c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')    

plt.rcParams['axes.unicode_minus'] = False

In [28]:
petition = pd.read_csv('Final BlueHouse Petition data.csv', index_col=0)
petition.head()

,제목,청원시작일,청원종료일,참여인원,URL,카테고리,청원내용,청원답변
0,방송통신위원회는 방송의 공적 책임 방기하고 언론이기를 포기한 채널A와 TV조선의 ...,2020. 04. 07,2020. 05. 07,273513,https://www1.president.go.kr/petitions/587827,문화/예술/체육/언론,최근 MBC의 잇따른 보도에 따르면 종편 채널A 기자가 현직 검사장과의 친분을 내세...,안녕하십니까. 방송통신위원회 위원장 한상혁입니다. 오늘 답변드릴 ‘채널A와 TV조선...
1,렌트카 훔쳐 사망사고를 낸 10대 엄중 처벌해주세요,2020. 04. 02,2020. 05. 02,1007040,https://www1.president.go.kr/petitions/587624,기타,29일 오전 0시 1분쯤 대전 동구의 한 네거리에서 훔친 렌터카를 몰던 10대 청소...,안녕하십니까. 디지털소통센터장 강정수입니다. 오늘은 「렌트카 훔쳐 사망사고 낸 ...
2,세월호 전면재수사,2020. 04. 01,2020. 05. 01,216118,https://www1.president.go.kr/petitions/587604,기타,저는 세월호 희생자 단원고 2학년 6반 순범이 큰 누나 입니다.6주기가 3주도 안남...,안녕하십니까. 시민참여비서관 이기헌입니다. 「세월호 참사 전면 재수사」국민청원에...
3,자가격리를 어기고 제주도 4박5일 여행.미국유학생 강남구 **번 확진자 처벌해주세요,2020. 03. 27,2020. 04. 26,207563,https://www1.president.go.kr/petitions/587334,행정,2주 자가격리를 어기고 제주도 4박5일 여행을 다녀온 강남구 **번 확진자와 같은 ...,안녕하십니까. 사회정책비서관 정동일입니다. 오늘은 <수출용 진단키트 독도 명칭 ...
4,수출용 코로나19 진단키트 이름을 독도로 해주세요,2020. 03. 25,2020. 04. 24,385617,https://www1.president.go.kr/petitions/587169,기타,지금 우리나라뿐 아니라 모든 나라가 코로나19사태로 어려움을 겪고 있습니다.우리나라...,안녕하십니까. 사회정책비서관 정동일입니다. 오늘은 <수출용 진단키트 독도 명칭 ...


In [5]:
petition.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 173
Data columns (total 8 columns):
제목       174 non-null object
청원시작일    174 non-null object
청원종료일    174 non-null object
참여인원     174 non-null int64
URL      174 non-null object
카테고리     174 non-null object
청원내용     174 non-null object
청원답변     171 non-null object
dtypes: int64(1), object(7)
memory usage: 12.2+ KB


In [22]:
petition['카테고리'].value_counts()

인권/성평등         46
정치개혁           24
기타             23
안전/환경          22
문화/예술/체육/언론    15
육아/교육          10
반려동물            8
보건복지            6
교통/건축/국토        5
외교/통일/국방        5
행정              3
미래              2
성장동력            2
경제민주화           2
일자리             1
Name: 카테고리, dtype: int64